The objective of the notebook is to randomly select 50k users to process a weekley time series for the following varibles: 

- Total number of transactions 
- Number of active days 
- Average transactions on weekdays 
- Average transactions on weekends

In [5]:
import os
import pandas as pd 
import numpy as np 

# 1. Read all the files to create a unique SMART_CARD_ID database

 TO DO: Clean data. The format that the data is stored is not standards. 
                    A cleaning procedure is important. For now I will assume that i 
                    I can read all files with the same format. 

 - Ideas to assert 
 1. Each month should have the days number of files. e.g.: If sept has 30 day, then it should have 30 files in the corresponding folder

In [19]:
path = '../../raw_data/2018_09_ValTroncal Sep2018'
files = os.listdir(path)

In [20]:
assert len(files) == 30, "September has 30 months"

In [34]:
id_list = []
for file in files[:3]:
    df = pd.read_csv(os.path.join(path, file), delimiter = ';')
    id_list.append(df['Numero de Tarjeta'])

ids = pd.concat(id_list)
unique_ids = ids.drop_duplicates()

print ('Number of transactions: {}'.format(len(ids)))
print ('Number of Unique IDS: {}'.format(len(unique_ids)))

Number of transactions: 3666486
Number of Unique IDS: 1719164


In [36]:
#Save file (I suspect that this code will take a long time to run)
unique_ids.to_csv('../output/unique_ids.csv', index = False)

# 2. Randomly select 50k smartcard 

In [40]:
sample_size = 50000
unique_ids = pd.read_csv('../output/unique_ids.csv')
sample_ids = unique_ids.sample(sample_size)

In [44]:
sample_ids

,Numero de Tarjeta
634662,2003010007541576
16055,1010000054131107
158582,1010000144571791
1418363,1010000059072918
1129417,1010000109514729
...,...
754811,1010000009997479
1220068,1010000004335923
509040,1010000139556948
534322,1010000145699518


# 3. Create a sinlge file with all the transactions of the 50k smartcard (in parquet maybe) 

In [86]:
transaction_list = []
for file in files[:15]:
    df = pd.read_csv(os.path.join(path, file), delimiter = ';')
    mask = sample_ids['Numero de Tarjeta']
    df_trans = df[df['Numero de Tarjeta'].isin(mask)]
    transaction_list.append(df_trans)
transactions = pd .concat(transaction_list)

In [106]:
# Clean transactions (and delete columns that are not relevant to save space)

#relevant_cols
cols = ['Fecha de Transaccion', 'Emisor', 'Estacion', 'Tipo de Tarjeta', 'Nombre de Perfil',
        'Numero de Tarjeta', 'Tipo de Tarifa', 'Saldo Previo a Transaccion', 'Valor']

transactions = transactions[cols]

In [107]:
fecha_transaction = transactions['Fecha de Transaccion'].astype(str)
transactions['timestamp'] = pd.to_datetime(fecha_transaction, format= '%Y%m%d%H%M%S')
# transactions['year'] = fecha_transaction.str[:4]
# transactions['month'] = fecha_transaction.str[4:6]  
# transactions['day'] = fecha_transaction.str[6:8]  
# transactions['minute'] = fecha_transaction.str[8:10]  
# transactions['seconds'] = fecha_transaction.str[10:12]  

# 4. Aggregate variables by week and create time series for each person

In [108]:
weekly_transactions = transactions.groupby(['Numero de Tarjeta',
                                            pd.Grouper(key = 'timestamp',freq='W')])['Emisor'].count()


weekly_transactions = weekly_transactions.reset_index()
weekly_transactions = pd.pivot_table(weekly_transactions, 
               values = 'Emisor', 
               index = 'Numero de Tarjeta', 
               columns = 'timestamp',
               aggfunc=np.sum)

weekly_transactions.fillna(0, inplace = True)
weekly_transactions

timestamp,2018-09-02,2018-09-09,2018-09-16,2018-09-23,2018-09-30
Numero de Tarjeta,,,,,
396750443,0.0,4.0,6.0,2.0,2.0
783476603,0.0,0.0,2.0,3.0,0.0
1122976996,1.0,3.0,5.0,1.0,3.0
1123040324,3.0,0.0,0.0,0.0,0.0
1123075108,0.0,2.0,0.0,2.0,0.0
...,...,...,...,...,...
54097253376793092,3.0,0.0,0.0,0.0,0.0
54099858936108292,0.0,3.0,2.0,3.0,0.0
54099858938528516,0.0,0.0,0.0,1.0,0.0
